In [1]:
import os,sys
import xarray as xr
import numpy as np
import functions as fun

In [2]:
#Load grid and fingerprints
with xr.open_dataset('../data/fingerprints.nc') as ds:
    lon = ds['lon'].values
    lat = ds['lat'].values
    fp_a = ds['fp_a'].values
    fp_gr = ds['fp_gr'].values
    fp_gl = ds['fp_gl'].values


In [3]:
def rsl(model,scen):
    #Get ocean dynamics
    with xr.open_dataset(f'../rawdata/zos_hist/CorrectedZOS_EXPrcp85_{model}.nc') as ds:
        ds = ds.sel(time=slice(1901,2100))
        odyn = ds['CorrectedZOS_reg'].values
        time = ds['time'].values

    with xr.open_dataset(f'../rawdata/zos/CorrectedZOS_EXP{scen}_{model}.nc') as ds:
        odyn = np.append(odyn,ds['CorrectedZOS_reg'].values,axis=0)
        time = np.append(time,ds['time'].values)
        
    #Add steric expansion    
    with xr.open_dataset(f'../data/steric/{model}_{scen}.nc') as ds:
        ds = ds.sel(time_lt=slice(1901,2100))
        slr = ds['SLR_lt'].values
    ster = slr[:,np.newaxis,np.newaxis]*np.ones((len(time),len(lat),len(lon)))
    ster[:,np.isnan(fp_a)] = np.nan
    
    #Add Antarctic dynamics
    with xr.open_dataset(f'../data/Adyn/{model}_{scen}.nc') as ds:
        ds = ds.sel(time_lt=slice(1901,2100))
        slr = ds['SLR_lt'].values
    adyn = slr[:,np.newaxis,np.newaxis]*fp_a[np.newaxis,:,:]
    
    with xr.open_dataset(f'../data/Asmb/{model}_{scen}.nc') as ds:  
        ds = ds.sel(time_lt=slice(1901,2100))
        slr = ds['SLR_lt'].values
    asmb = slr[:,np.newaxis,np.newaxis]*fp_a[np.newaxis,:,:]
    
    #Add Greenland smb
    with xr.open_dataset(f'../data/Gsmb/{model}_{scen}.nc') as ds:  
        ds = ds.sel(time_lt=slice(1901,2100))
        slr = ds['SLR_lt'].values
    gsmb = slr[:,np.newaxis,np.newaxis]*fp_gr[np.newaxis,:,:]
    
    #Add glaciers
    for r in range(1,20):
        with xr.open_dataset(f'../data/glacier/{model}_{scen}_{r}.nc') as ds:
            ds = ds.sel(time_lt=slice(1901,2100))
            slr = ds['SLR_lt'].values
        if r==1:
            glac = slr[:,np.newaxis,np.newaxis]*fp_gl[r-1,:,:]
        else:
            glac += slr[:,np.newaxis,np.newaxis]*fp_gl[r-1,:,:]

    #Add others
    with xr.open_dataset(f'../data/others/gdyn_{scen}.nc') as ds:
        gdyn = ds['SLR_lt'].values
    with xr.open_dataset(f'../data/others/grw.nc') as ds:
        grw = ds['SLR_lt'].values    
    with xr.open_dataset(f'../data/others/gia.nc') as ds:
        gia = ds['SLR_lt'].values
    
    #Get total rsl in cm
    slr = odyn+ster+adyn+asmb+gsmb+glac+gdyn+grw+gia
    
    #Save
    slr2 = xr.DataArray(slr,dims=('time','lat','lon'),coords={'time':time,'lat':lat,'lon':lon})
    ds = xr.Dataset({'SLR':slr2})
    ds.to_netcdf(f'../data/totalrsl/{model}_{scen}.nc',encoding={'SLR':{'zlib': True, 'complevel': 9}})
    ds.close()
    
    print(scen,model,'Saved rsl')

In [4]:
mods = fun.models()
mods = ['bcc-csm1-1','CanESM2','CCSM4','CNRM-CM5','CSIRO-Mk3-6-0','GFDL-CM3','GISS-E2-R','HadGEM2-ES','inmcm4','IPSL-CM5A-LR','MIROC-ESM','MIROC5','MPI-ESM-LR','MRI-CGCM3','NorESM1-M']
for model in mods:
    for scen in ['rcp45','rcp85']:
        rsl(model,scen) 

rcp45 bcc-csm1-1 Saved rsl
rcp85 bcc-csm1-1 Saved rsl
rcp45 CanESM2 Saved rsl
rcp85 CanESM2 Saved rsl
rcp45 CCSM4 Saved rsl
rcp85 CCSM4 Saved rsl
rcp45 CNRM-CM5 Saved rsl
rcp85 CNRM-CM5 Saved rsl
rcp45 CSIRO-Mk3-6-0 Saved rsl
rcp85 CSIRO-Mk3-6-0 Saved rsl
rcp45 GFDL-CM3 Saved rsl
rcp85 GFDL-CM3 Saved rsl
rcp45 GISS-E2-R Saved rsl
rcp85 GISS-E2-R Saved rsl
rcp45 HadGEM2-ES Saved rsl
rcp85 HadGEM2-ES Saved rsl
rcp45 inmcm4 Saved rsl
rcp85 inmcm4 Saved rsl
rcp45 IPSL-CM5A-LR Saved rsl
rcp85 IPSL-CM5A-LR Saved rsl
rcp45 MIROC-ESM Saved rsl
rcp85 MIROC-ESM Saved rsl
rcp45 MIROC5 Saved rsl
rcp85 MIROC5 Saved rsl
rcp45 MPI-ESM-LR Saved rsl
rcp85 MPI-ESM-LR Saved rsl
rcp45 MRI-CGCM3 Saved rsl
rcp85 MRI-CGCM3 Saved rsl
rcp45 NorESM1-M Saved rsl
rcp85 NorESM1-M Saved rsl
